## Transactions
#### 03.4 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2024-02-14 (started 2021-12-19)

- Load credentials
- Connect to the blockchain
- Execute a first payment transaction using Python
- Add a note to the payment

## Setup
Starting with this chapter 3.4, the lines below will always automatically load ...
* The accounts MyAlgo, Alice, Bob, Charlie, Dina
* The API credentials
* The functions in `algo_util.py`
    * These functions can be found in the folder `sharedCode` which is two levels up

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
import algosdk.error
import json

In [ ]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

## Connecting to the Algorand Blockchain via API
+ We choose to connect via Algonode API
- API **address** stored ...
    - For the testnet in `cred['algod_test']`
    - For the mainnet in `cred['algod_main']`
- API **crendentials** stored in `cred['api_token']` (currently not needed)

In [ ]:
# Today we work with the testnet
algod_token   = ''                        # Only needed if we have our own server
algod_address = cred['algod_test']        # TESTNET, alternatively cred['algod_main'] 
api_token = cred['api_token']             # Currently empty, this may change

# Initialize the algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address, headers=api_token)

#### Test the connection
- Our first Python access of the blockchain
- What's the last block?
    - Note that block count on testnet is larger (Why?)
    - Check here (select *testnet* on top left): https://app.dappflow.org/explorer/home

In [ ]:
algod_client.status()["last-round"]

### ❗️ Mainnet or testnet?
* Both networks are identical from a technical point of view
* Private and public keys are eqally valid on both nets
    * Every main net account has a "twin" on the testnet
    * Treat testnet credentials with same care as mainnet credentials
* Tokens on main net have economic value
* On the testnet, ALGOs and USDC can be obtained for free
* Use testnet to learn without having to pay transaction fees

**Connect to mainnet or testnet**
* To select mainnet or testnet, change just one line when creating the `algod_client()`:

```
algod_address = cred['algod_test']    # TESTNET, for mainnet use cred['algod_main'] 
```

### Check the accounts on the blockchain
#### (a) on the Mainnet
- Check the link below to find the Algos that you have received at the beginning of class

In [ ]:
# Create a link to directly access your MyAlgo account
print(cred['explore_main']+'address/'+MyAlgo['public'])

#### (b) on the Testnet
* Check the link below to verify that `MyAlgo` account on the testnet has still 0 Algos *on the testnet*

In [ ]:
print(cred['explore_test']+'address/'+MyAlgo['public'])

### Fund with testnet Algos

Try any of these:
- https://bank.testnet.algorand.network/
- https://dispenser.testnet.aws.algodev.network
- Fund `MyAlgo`, `Alice` and `Bob`. How many test ALGOs did you get?

### Obtain holdings
* The `algod_client.account_info()` function obtains all sorts of information about an account
    * For more details, see chapter 11 (empirics)

In [ ]:
# Get holdings of testnet Algos
address = Alice["public"]
algod_client.account_info(address)["amount"]

In [ ]:
# Holdings are in micro Algo ... convert
algo_precision = 1e6
algo_amount    = algod_client.account_info(address)["amount"]/algo_precision
print(f"Address {address} has {algo_amount} test algos")

## A first payment transaction

#### The suggested parameters for a transaction (on the test network)
* Get standard parameters from API
* Simplifies creation of a transaction
* Part of ever y**Step 1 – prepare transaction**
    * `first` and `last` are suggested params for first and last round valid. Compare to current round
    * Suggested fee is standard fee
    * `gh` stands for genesis hash (=first entry on the blockchain)

In [ ]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

#### Step 1: prepare and create transaction using `PaymentTxn`

In [ ]:
# Parameters
sp        = algod_client.suggested_params()       # suggested params
amount    = 0.1                                   # in ALGO
algo_prec = 1e6
amt_microalgo = int(amount * algo_prec)     # in Micro-ALGO 

# Create (unsigned) TX
txn = PaymentTxn(sender = Alice['public'],     # <--- From
                 sp = sp,                      # <---- ALWAYS include the sp
                 receiver = Bob['public'],     # <---- To
                 amt = amt_microalgo           # <---- in Micro-ALGO
                )

In [ ]:
# Interesting: this is how a transaction looks like
print(json.dumps(vars(txn), indent=4))

In [ ]:
# Interesting: we already have a txid
print(txn.get_txid())

In [ ]:
# Is it already on the blockchain? 
# No ... we have not yet sent sent it --> 404 error
print(cred['explore_test']+'tx/'+txn.get_txid())

#### Step 2: sign

In [ ]:
stxn = txn.sign(Alice['private'])                 # <---- Alice signs with private key

In [ ]:
# Interesting: this is how a transaction looks like
# The new stxn object consists of
print(stxn.transaction)                           # same as above
print('')
print(stxn.signature)                             # signature signs the transaction

In [ ]:
# Interesting: The transaction ID is the same as before
# ... and still nothing on the blockchain
print(stxn.get_txid())
print(cred['explore_test']+'tx/'+stxn.get_txid())

#### Step 3: send

In [ ]:
txid = algod_client.send_transaction(stxn)

In [ ]:
# Transaction just asent to the blockchain
# Does not yet contain a `confirmed-round` object
txinfo = algod_client.pending_transaction_info(txid)
print(txinfo)

#### Step 4: Wait for confirmation

In [ ]:
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
# Note that txinfo has now a 'confirmed-round'
print(txinfo)

In [ ]:
# Now we can check the tx also on Pera Explorer
print(cred['explore_test']+'tx/'+txid)

## Add a note to a transaction
* Create a transaction a bit more efficiently
* Add a transaction note

In [ ]:
# Step 1a: Prepare
sp     = algod_client.suggested_params()       # suggested params
amount = 0.1
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Step 1b: The note (need to encode as bytes)
note_txt  = "Paying back for last dinner"
note_byte = note_txt.encode()

In [ ]:
# Step 1c: create (unsigned) TX
txn = PaymentTxn(sender=Alice['public'],
                 sp = sp,
                 receiver = Bob['public'],
                 amt=amt_microalgo, 
                 note=note_byte
                 )
print(txn)

In [ ]:
# Step 2+3: sign and send TX
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
print(txid)

In [ ]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
print(txinfo)

In [ ]:
# Check on Pera Explorer
# The note is readable in plain text
print(cred['explore_test']+'tx/'+txid)

### Step 5 (check): Extract message in txinfo and convert back to plain text

In [ ]:
import base64
note_base64 = txinfo['txn']['txn']['note']
print(note_base64)
note_byte   = base64.b64decode(note_base64)
print(note_byte)
note_txt   = note_byte.decode()
print(note_txt)

## Exercises

**Exercise 1:** Send 0.82 ALGO from Dina to Alice with a thank you note

In [ ]:
# Your Python code goes here

# Step 1: Prepare
sp        = algod_client.suggested_params()       # suggested params
amount    = 0.82
algo_prec = 1e6
amt_microalgo = int(amount * algo_prec)
note_txt  = "Thank you"
note_byte = note_txt.encode()

txn = PaymentTxn(sender=Dina['public'],
                 sp=sp, 
                 receiver = Alice['public'],
                 amt=amt_microalgo, 
                 note=note_byte
                 )

# Step 2+3: sign and send TX
stxn = txn.sign(Dina['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

**Exercise 2:** Obtain the (new) ALGO holdings of Alice using Python

In [ ]:
# Your Python code goes here
address = Alice["public"]
algod_client.account_info(address)["amount"] / 1e6

## Things that do not and will not work
* Following are a few things that deliberately don't work.
* Goal: learn how error messages look like and how to deal with them.

In [ ]:
# Need to import this to be able to read error messages
import sys, algosdk.error

### Overspending
* Alice sends more than she owns
* The error message is very long. Scroll down to the end.

In [ ]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [ ]:
# Step 3b: Send
txid = algod_client.send_transaction(signed_txn)

#### Can we *catch the error* and get a better structured error message?
* **Note:** error occurs when sending the transaction

In [ ]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [ ]:
# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                   # print entire error message
    if ("overspend" in str(err)):                # check for specific type of error
        print("Overspend error")         
    txid = None

#### What happens if we wait for the failed transaction to complete?

In [ ]:
# Step 4: Wait for confirmation
try:
    txinfo = wait_for_confirmation(algod_client, txid)
    print(txinfo)
except TypeError as err:                                       # obtain error message
    # print entire error message (rather cryptic!)
    print(err)
    # Give better error message
    print("txid is empty")

### Wrong signature
Bob tries to sign a transaction from Alice to Bob

In [ ]:
# Step 1: prepare
sp        = algod_client.suggested_params()
algo_prec = 1e6
sender    = Alice['public']
receiver  = Bob['public']
amount    = 0.1
amount_microalgo = int(amount * algo_prec)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Bob['private'])                # <----------------- wrong person signs!

In [ ]:
# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("should have been authorized" in str(err)):                # check for specific type of error
        print("Wrong signature error")         
    txid = None

### *Sending the *indentical* transaction twice
* It is not possible to send the *identical* transaction twice
    * Reason: the transaction ID is calculated from the transaction data
* "Identical" means ...
    * same Sender
    * same Recipient
    * same Ammount
    * same (suggested) parameters (including first/last round) $\leftarrow$ change after 2-3 seconds

In [ ]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 0.1
amount_microalgo = int(amount * algo_precision)

In [ ]:
# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
    print("Submitted with txID: {}".format(txid))
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("transaction already in ledger" in str(err)):                # check for specific type of error
        print("Identical transaction {} has been submitted twice.".format(signed_txn.get_txid()))         
    txid = None    # check for specific type of error

**REPEAT** only step 2-3 $\rightarrow$ error message<br>
**REPEAT** only step 1-3 $\rightarrow$ no error <br>

#### See how the sp change
* Re-run this after 2-3 seconds

In [ ]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

## Appendix: Useful functions
* The following functions are included in `algo_util.py`

In [ ]:
def wait_for_confirmation(client, txid):
    # client = algosdk client
    # txid = transaction ID, for example from send_payment()
    # An ufficial Algorand function

    txinfo = client.pending_transaction_info(txid)       # obtain transaction information
    current_round = algod_client.status()["last-round"]        # obtain last round number
    print("Current round is  {}.".format(current_round))
    
    # Wait for confirmation
    while ( txinfo.get('confirmed-round') is None ):            # condition for waiting = 'confirmed-round' is empty
        print("Waiting for round {} to finish.".format(current_round))
        algod_client.status_after_block(current_round)             # this wait for the round to finish
        txinfo = algod_client.pending_transaction_info(txid)    # update transaction information
        current_round += 1

    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

### *Python dict as transaction note
* For attaching a Python dict as message to a note
* Example `note_dict = {'from' : 'Bob', 'to' : 'Alice', 'message' : 'Many thanks'}

In [ ]:
def note_encode(note_dict):
    # note dict ... a Python dictionary
    note_json = json.dumps(note_dict)
    note_byte = note_json.encode()     
    return(note_byte)

In [ ]:
def note_decode(note_64):
    # note64 =  note in base64 endocing
    # returns a Python dict
    import base64
    message_base64 = txinfo['txn']['txn']['note']
    message_byte   = base64.b64decode(message_base64)
    message_json   = message_byte.decode()
    message_dict   = json.loads( message_json )
    return(message_dict)